In [25]:
import pandas as pd
from collections import defaultdict

# ---------- Display (optional) ----------
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', None)

# ---------- Global State ----------

portfolio_state = {
    'cash': 200,                     # constant after reset
    'remaining': 200,                # evolves with trades
    'quantities': defaultdict(int),  # {ticker: quantity}
    'cost_basis': defaultdict(float),# {ticker: absolute cost basis}
    'avg_price': defaultdict(float), # {ticker: average entry price}
    'realized_pnl': 0.0,
    'last_price': {},                # {ticker: last seen price}
    'portfolio_df': None             # accumulator
}

# Track trade numbers per ticker
trade_tracker = {}  # {ticker: trade_number}
next_trade_number = 1  # Global counter for next new trade

COLUMNS = [
    'Date','Ticker','Buy/Sell','Position Taken','Current Position','Cash','Buyable/Sellable',
    'Quantity Buy','Remaining','Current Quantity','Price',
    'Avg Price','Cost Basis','Position Value PV',
    'PnL (Long) Unrealized','PnL (Short) Unrealized','Pnl Unrealized','PnL Unrealized Value',
    'PV (Long)','PV (Short)','Open Position','Open PV',
    'Total PV','Equity: Total PV + Remaining','PnL Realized','Total PnL Overall (Unrealized+Realized)',
    'Daily PnL (Unrealized+Realized)','Liquidation Price','Take Profit','Stop Loss', 
    'Last Day Pnl / Daily $', 'Daily %', 'Cumulative %', 'Performance', 
    'Trade No. (Position - Trade no. - Current Quantity)', 'Total Trades', 'Win/Loss', 'Win Rate', 'Win:Loss Ratio', 'Trades/Month'
]

# ---------- Lifecycle ----------

def reset_portfolio(initial_cash=200):
    global portfolio_state, trade_tracker, next_trade_number
    portfolio_state = {
        'cash': initial_cash,
        'remaining': initial_cash,
        'quantities': defaultdict(int),
        'cost_basis': defaultdict(float),
        'avg_price': defaultdict(float),
        'realized_pnl': 0.0,
        'last_price': {},
        'portfolio_df': pd.DataFrame(columns=COLUMNS)
    }
    trade_tracker = {}
    next_trade_number = 1

def get_portfolio_df():
    return portfolio_state['portfolio_df'].copy()

# ---------- Helpers ----------

def normalize_quantity(q):
    """
    Accepts:
      - numeric: 10, -5
      - strings: "-(-10)" -> 10,  "(10)" -> 10, "-10" -> -10, "  -(-5)  " -> 5
    """
    if isinstance(q, (int, float)):
        return float(q)
    s = str(q).strip().replace(' ', '')
    if s.startswith('-(') and s.endswith(')'):
        inner = s[2:-1]
        return -float(inner)
    if s.startswith('(') and s.endswith(')'):
        inner = s[1:-1]
        return float(inner)
    return float(s)

def get_or_create_trade_number(ticker, old_q, new_q, action):
    """
    Determine trade number for a ticker based on position state.
    - If old_q == 0 and new_q != 0: Opens new trade (assign new trade number)
    - If old_q != 0 and new_q != 0: Continues existing trade (keep same trade number)
    - If old_q != 0 and new_q == 0: Closes existing trade (keep same trade number, mark as closed)
    - If old_q == 0 and new_q == 0: No trade
    """
    global next_trade_number
    ticker = str(ticker).upper()
    
    action_lower = str(action).lower()
    
    # If action is 'hold', check if position exists
    if action_lower == 'hold':
        if ticker in trade_tracker and old_q != 0:
            # Position exists but holding, return existing trade number
            return trade_tracker[ticker]
        else:
            return None
    
    # If opening a new position (old_q == 0, new_q != 0)
    if old_q == 0 and new_q != 0:
        # Assign new trade number
        trade_number = next_trade_number
        trade_tracker[ticker] = trade_number
        next_trade_number += 1
        return trade_number
    
    # If closing a position (old_q != 0, new_q == 0)
    if old_q != 0 and new_q == 0:
        # Return existing trade number, then remove from tracker
        trade_number = trade_tracker.get(ticker)
        if ticker in trade_tracker:
            del trade_tracker[ticker]
        return trade_number
    
    # If maintaining/changing position (old_q != 0, new_q != 0)
    if old_q != 0 and new_q != 0:
        # Continue existing trade
        return trade_tracker.get(ticker)
    
    # No position (old_q == 0, new_q == 0)
    return None

def format_trade_string(action, current_position, trade_number, new_q):
    """
    Format trade string: "Buy/Sell/Short - #TradeNo Trade - Quantity" or "... - 0 - close"
    Includes Buy/Sell action and uses current_position instead of position
    """
    if trade_number is None:
        return "No Buy/Sell"
    
    action_str = str(action)
    pos = str(current_position)
    
    # Format quantity - use absolute value since position already explains it
    if new_q == 0:
        quantity_str = "0 - Close"
    else:
        # Use absolute value for quantity
        quantity_str = str(int(abs(new_q)))
    
    # Include Buy/Sell in the output
    return f" {pos.capitalize()} - {action_str.capitalize()} - #{trade_number} Trade - {quantity_str}"

# ---------- Core Single-Trade Calculations ----------

def calculate_cash_single():
    # Cash is constant after initialization
    return portfolio_state['cash']

def calculate_remaining_single(action, position, price, q_in, old_quantity, old_cost_basis):
    """
    Remaining update (prev = prior Remaining, p = current price, q = shares):
    - Buy Long:   prev - p*q
    - Sell Long:  prev + p*q
    - Sell Short: prev - p*q
    - Buy Short:  prev + [ initial + (initial - final) ]
                  where initial = avg * close_qty, final = p * close_qty
    
    Uses previous avg price (from old_cost_basis / old_quantity) for closing calculations.
    
    Position flips automatically when quantity crosses zero:
    - Long selling more than owned: closes long, opens short with excess
    - Short buying more than owed: covers short, opens long with excess
    """
    rem = portfolio_state['remaining']
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in
    if qty == 0 or a == 'hold':
        return rem

    # Previous avg price (0 if no position)
    prev_avg = abs(old_cost_basis / old_quantity) if old_quantity else 0.0

    if a == 'buy':
        if old_quantity < 0:
            # Buy Short (cover up to held short) using prev_avg for initial
            cover = min(qty, abs(old_quantity))
            if cover > 0:
                initial = prev_avg * cover
                final = price * cover
                delta = initial + (initial - final)
                rem += delta
            # Any excess turns into Buy Long at market
            excess = qty - cover
            if excess > 0:
                rem -= price * excess
        else:
            # Buy Long
            rem -= price * qty
        return rem

    if a == 'sell':
        if old_quantity > 0:
            # Sell Long up to held long
            close = min(qty, old_quantity)
            if close > 0:
                rem += price * close
            # Any excess turns into Sell Short at market
            excess = qty - close
            if excess > 0:
                rem -= price * excess
        else:
            # Sell Short (open/increase short)
            rem -= price * qty
        return rem

    return rem

def calculate_current_quantity_single(ticker, action, position, q_in, old_quantity):
    """
    Quantity change depends on action:
    - Buy: increases quantity (moves longward)
    - Sell: decreases quantity (moves shortward)
    
    Position flips automatically when quantity crosses zero:
    - Long selling more than owned → flips to short
    - Short buying more than owed → flips to long
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in

    if a == 'hold':
        new_q = old_quantity
    elif a == 'buy':
        new_q = old_quantity + qty  # Naturally flips short→long if buying excess
    elif a == 'sell':
        new_q = old_quantity - qty  # Naturally flips long→short if selling excess
    else:
        new_q = old_quantity

    portfolio_state['quantities'][ticker] = new_q
    return new_q

def calculate_avg_price_and_cost_basis_single(ticker, action, position, price, q_in, old_quantity, new_quantity, old_cost_basis):
    """
    Maintain absolute Cost Basis and Avg Price for net position.
    Conventions:
    - Long: Cost Basis = dollars spent on current net long shares
    - Short: Cost Basis = dollars received from current net short shares (short proceeds)
    - Avg Price = abs(Cost Basis / Quantity) when quantity != 0
    - Crossing 0: prior side closes, new side starts fresh
    """
    a = str(action).lower()
    qty = abs(q_in) if q_in < 0 else q_in
    cb = old_cost_basis

    if a == 'hold':
        pass

    elif a == 'buy':
        if old_quantity >= 0:
            # adding/opening long
            cb = cb + qty * price
        else:
            # buying to cover short
            to_cover = min(qty, abs(old_quantity))
            if qty > to_cover:
                # fully cover then open long with residual
                open_long = qty - to_cover
                cb = open_long * price
            else:
                # still short; proportionally reduce short proceeds
                if abs(old_quantity) > 0:
                    remaining_short = abs(old_quantity) - to_cover
                    cb = cb * (remaining_short / abs(old_quantity)) if remaining_short > 0 else 0.0

    elif a == 'sell':
        if old_quantity > 0:
            # selling from long
            sell_qty = qty
            if sell_qty < old_quantity:
                remaining_long = old_quantity - sell_qty
                cb = cb * (remaining_long / old_quantity)
            elif sell_qty == old_quantity:
                cb = 0.0
            else:
                # flipped to short: close long then open short with extra
                open_short = sell_qty - old_quantity
                cb = open_short * price
        else:
            # short selling or increasing short
            cb = cb + qty * price

    if new_quantity != 0:
        avg_price = abs(cb / new_quantity)
    else:
        avg_price = 0.0
        cb = 0.0

    portfolio_state['cost_basis'][ticker] = cb
    portfolio_state['avg_price'][ticker] = avg_price
    return avg_price, cb

def calculate_realized_pnl_single(ticker, action, position, price, q_in, old_quantity):
    """
    Realized PnL on closing legs - cumulative across ALL tickers.
    - Closing long by selling: (sell_price - avg_entry) * shares_closed
    - Covering short by buying: (avg_entry - cover_price) * shares_closed
    
    Uses previous avg price from state (reads directly before updating).
    Long position: calculate realized PnL when action is 'sell'
    Short position: calculate realized PnL when action is 'buy'
    
    IMPORTANT: When position flips (selling more than owned or buying more than owed),
    only the closed portion generates realized PnL, not the excess.
    """
    realized = portfolio_state['realized_pnl']
    a = str(action).lower()
    pos = str(position).lower()
    
    # Read old avg price from state (before it gets updated)
    prev_avg_price = portfolio_state['avg_price'][ticker]

    # Long position: calculate realized PnL when selling
    if pos == 'long' and a == 'sell' and old_quantity > 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually closed (min of qty and owned)
        # If selling more than owned, only the owned shares generate realized PnL
        closed = min(qty, old_quantity)
        if closed > 0 and prev_avg_price > 0:
            realized += (price - prev_avg_price) * closed

    # Short position: calculate realized PnL when buying/covering
    if pos == 'short' and a == 'buy' and old_quantity < 0:
        qty = abs(q_in) if q_in < 0 else q_in
        # Only realize on shares actually covered (min of qty and owed)
        # If buying more than owed, only the owed shares generate realized PnL
        closed = min(qty, abs(old_quantity))
        if closed > 0 and prev_avg_price > 0:
            realized += (prev_avg_price - price) * closed

    portfolio_state['realized_pnl'] = realized
    return realized

# ---------- Derived per-trade ----------

def position_value_from_position(position, new_quantity, price):
    pos = str(position).lower()
    if pos == 'short':
        # show positive PV for shorts
        return abs(new_quantity) * price
    # long/hold → normal signed PV
    return new_quantity * price

def pnl_unrealized_components(new_quantity, price, avg_price):
    if new_quantity > 0 and avg_price > 0:
        long_u = (price - avg_price) * new_quantity
    else:
        long_u = 0.0

    if new_quantity < 0 and avg_price > 0:
        short_u = (avg_price - price) * abs(new_quantity)
    else:
        short_u = 0.0

    return long_u, short_u, (long_u + short_u)

def open_positions_str():
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {q}")
    return ", ".join(parts) if parts else "None"

def open_pv_str(current_ticker, current_price, current_position):
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            cb = portfolio_state['cost_basis'][t]
            avg = portfolio_state['avg_price'][t]
            
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            if q > 0 and avg > 0:
                long_u = (p - avg) * q
                ticker_pv = cb + long_u
            elif q < 0 and avg > 0:
                short_u = (avg - p) * abs(q)
                ticker_pv = cb + short_u
            else:
                ticker_pv = 0.0
            
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {ticker_pv}")
    return ", ".join(parts) if parts else "None"

def calculate_pv_for_current_ticker(current_ticker, current_price, current_position, new_q, avg_p, cb):
    """
    Calculate PV (Long) and PV (Short) ONLY for the current ticker being traded.
    PV = Cost Basis + Unrealized PnL
    """
    pos = str(current_position).lower()
    
    if pos == 'long' and new_q > 0 and avg_p > 0:
        long_u = (current_price - avg_p) * new_q
        pv_long = cb + long_u
        return pv_long, 0.0
    elif pos == 'short' and new_q < 0 and avg_p > 0:
        short_u = (avg_p - current_price) * abs(new_q)
        pv_short = cb + short_u
        return 0.0, pv_short
    else:
        return 0.0, 0.0

def calculate_total_pv_all_tickers(current_ticker, current_price, current_position):
    """
    Calculate Total PV = sum of all long PVs + sum of all short PVs
    across all tickers in the portfolio (cumulative).
    PV = Cost Basis + Unrealized PnL for each position
    """
    total_long_pv = 0.0
    total_short_pv = 0.0
    
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            # Get cost basis and avg price for this ticker
            cb = portfolio_state['cost_basis'][t]
            avg = portfolio_state['avg_price'][t]
            
            # Determine current price
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            # Calculate unrealized PnL and PV
            if q > 0 and avg > 0:  # Long position
                long_u = (p - avg) * q
                ticker_pv_long = cb + long_u
                total_long_pv += ticker_pv_long
            elif q < 0 and avg > 0:  # Short position
                short_u = (avg - p) * abs(q)
                ticker_pv_short = cb + short_u
                total_short_pv += ticker_pv_short
    
    return total_long_pv + total_short_pv

def open_pnl_unrealized_str(current_ticker, current_price, current_position):
    parts = []
    for t, q in portfolio_state['quantities'].items():
        if q != 0:
            avg = portfolio_state['avg_price'][t]
            
            if t == current_ticker:
                p = current_price
            else:
                p = portfolio_state['last_price'].get(t, 0.0)
            
            if q > 0 and avg > 0:
                ticker_unrealized = (p - avg) * q
            elif q < 0 and avg > 0:
                ticker_unrealized = (avg - p) * abs(q)
            else:
                ticker_unrealized = 0.0
            
            ticker_upper = str(t).upper()
            parts.append(f"{ticker_upper} {ticker_unrealized}")
    
    return ", ".join(parts) if parts else "None"

def calculate_liquidation_price(current_position, new_q, avg_p):
    """
    Calculate liquidation price based on current position at that point in time.
    - Long position: liquidation at price = 0
    - Short position: liquidation at price = 2 * avg_price (100% loss = liquidated)
    - Hold position or quantity = 0: None (no position to liquidate)
    """
    pos = str(current_position).lower()
    
    # Check that we actually have a position (quantity != 0)
    if new_q == 0:
        return None
    
    if pos == 'long' and avg_p > 0:
        # Long position: liquidation at price = 0
        return 0.0
    elif pos == 'short' and avg_p > 0:
        # Short position: liquidation at price = 2 * avg_price
        # (e.g., short at 100, if price goes to 200 = 100% loss = liquidated)
        return 2.0 * avg_p
    else:
        # Hold position or invalid avg_p
        return None

def calculate_take_profit(current_position, new_q, avg_p, take_profit_pct=0.20):
    """
    Calculate Take Profit price based on current position.
    - Long: Avg Entry Price * (1 + Percentage)
    - Short: Avg Entry Price * (1 - Percentage)
    """
    if new_q == 0 or avg_p <= 0:
        return None
    
    pos = str(current_position).lower()
    
    if pos == 'long':
        # Long: Take Profit = Avg Price * (1 + Percentage)
        return avg_p * (1 + take_profit_pct)
    elif pos == 'short':
        # Short: Take Profit = Avg Price * (1 - Percentage)
        return avg_p * (1 - take_profit_pct)
    else:
        return None

def calculate_stop_loss(current_position, new_q, avg_p, stop_loss_pct=0.10):
    """
    Calculate Stop Loss price based on current position.
    - Long: Avg Entry Price * (1 - Percentage) [unrealized = -cost basis * percentage]
    - Short: Avg Entry Price * (1 + Percentage) [vice versa of TP]
    """
    if new_q == 0 or avg_p <= 0:
        return None
    
    pos = str(current_position).lower()
    
    if pos == 'long':
        # Long: Stop Loss = Avg Price * (1 - Percentage)
        return avg_p * (1 - stop_loss_pct)
    elif pos == 'short':
        # Short: Stop Loss = Avg Price * (1 + Percentage) [vice versa of TP]
        return avg_p * (1 + stop_loss_pct)
    else:
        return None

def calculate_trade_win_loss(trade_string, realized_pnl, previous_realized_pnl):
    """
    Calculate if a closed trade is a win or loss based on trade string.
    Win: realized PnL change > 0
    Loss: realized PnL change <= 0
    Only applies when trade closes (trade_string contains "- close")
    Checks the realized PnL column value for that row
    """
    # Check if trade is closing based on trade string format
    if trade_string == "No Buy/Sell" or "- close" not in trade_string:
        return None
    
    # Trade is closing, check the realized PnL change for this closing action
    realized_pnl_change = realized_pnl - previous_realized_pnl
    
    if realized_pnl_change > 0:
        return "Win"
    elif realized_pnl_change <= 0:
        return "Loss"
    else:
        return None

def calculate_win_rate(previous_df, current_win_loss):
    """
    Calculate win rate at this point in time.
    Win Rate = (Win values / Total non-None values) * 100
    Counts all previous Win/Loss values plus current one
    """
    # Get all previous Win/Loss values
    if len(previous_df) > 0:
        previous_win_loss = previous_df['Win/Loss'].tolist()
        # Add current win_loss to the list
        all_win_loss = previous_win_loss + [current_win_loss]
    else:
        all_win_loss = [current_win_loss]
    
    # Count non-None values
    non_none_values = [v for v in all_win_loss if v is not None]
    total_non_none = len(non_none_values)
    
    # Count Win values
    win_count = sum(1 for v in non_none_values if v == "Win")
    
    # Calculate win rate
    if total_non_none > 0:
        win_rate = (win_count / total_non_none) * 100
        return win_rate
    else:
        return None

def calculate_win_loss_ratio(previous_df, current_win_loss):
    """
    Calculate win:loss ratio at this point in time.
    Win:Loss Ratio = "win_count:loss_count" format (e.g., "1:0", "2:1", "0:1")
    """
    # Get all previous Win/Loss values
    if len(previous_df) > 0:
        previous_win_loss = previous_df['Win/Loss'].tolist()
        # Add current win_loss to the list
        all_win_loss = previous_win_loss + [current_win_loss]
    else:
        all_win_loss = [current_win_loss]
    
    # Count non-None values
    non_none_values = [v for v in all_win_loss if v is not None]
    
    # Count Win and Loss values
    win_count = sum(1 for v in non_none_values if v == "Win")
    loss_count = sum(1 for v in non_none_values if v == "Loss")
    
    # Return ratio in "win_count:loss_count" format
    return f"{win_count}:{loss_count}"

def calculate_trades_per_month(previous_df, current_date, current_trade_string):
    """
    Calculate number of open trades in the current month.
    Returns format: "count (Month Name)" or just count with month name
    """
    from datetime import datetime
    
    if current_date is None:
        return None
    
    # Parse current date and get month name
    try:
        if isinstance(current_date, str):
            # Try common date formats
            for fmt in ['%m/%d/%Y', '%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y']:
                try:
                    current_dt = datetime.strptime(current_date, fmt)
                    break
                except:
                    continue
            else:
                return None
        else:
            current_dt = pd.to_datetime(current_date)
        
        month_name = current_dt.strftime('%B')  # Full month name (e.g., "October")
        current_month_year = (current_dt.year, current_dt.month)
    except:
        return None
    
    # Get all rows including current
    all_data = []
    if len(previous_df) > 0:
        for _, row in previous_df.iterrows():
            all_data.append({
                'Date': row.get('Date'),
                'Trade String': row.get('Trade No. (Position - Trade no. - Current Quantity)', '')
            })
    all_data.append({
        'Date': current_date,
        'Trade String': current_trade_string
    })
    
    # Find open trades in current month
    open_trade_nums = set()
    
    for row_data in all_data:
        row_date = row_data['Date']
        trade_str = row_data['Trade String']
        
        if row_date is None or trade_str == "No Buy/Sell":
            continue
        
        # Check if row is in current month
        try:
            if isinstance(row_date, str):
                for fmt in ['%m/%d/%Y', '%Y-%m-%d', '%d/%m/%Y', '%m-%d-%Y']:
                    try:
                        row_dt = datetime.strptime(row_date, fmt)
                        break
                    except:
                        continue
                else:
                    continue
            else:
                row_dt = pd.to_datetime(row_date)
            
            row_month_year = (row_dt.year, row_dt.month)
            if row_month_year != current_month_year:
                continue
            
            # Extract trade number
            import re
            match = re.search(r'#(\d+)', str(trade_str))
            if match:
                trade_num = int(match.group(1))
                # Check if trade is closed
                if "- close" not in str(trade_str):
                    open_trade_nums.add(trade_num)
                else:
                    # Remove if closed
                    open_trade_nums.discard(trade_num)
        except:
            continue
    
    count = len(open_trade_nums)
    return f"{count} ({month_name})"

# ---------- Main entry per trade ----------

def process_trade(ticker, action, position, price, quantity_buy, date=None, take_profit_pct=0.20, stop_loss_pct=0.10):
    ticker = str(ticker).strip().upper()
    
    q_in = normalize_quantity(quantity_buy)

    old_q = portfolio_state['quantities'][ticker]
    old_cb = portfolio_state['cost_basis'][ticker]

    cash = calculate_cash_single()
    new_remaining = calculate_remaining_single(action, position, price, q_in, old_q, old_cb)

    new_q = calculate_current_quantity_single(ticker, action, position, q_in, old_q)
    
    current_position = 'short' if new_q < 0 else 'long' if new_q > 0 else 'hold'
    
    # Get realized PnL before updating to calculate the change
    realized_pnl_before = portfolio_state['realized_pnl']
    
    realized_pnl = calculate_realized_pnl_single(
        ticker, action, position, price, q_in, old_q
    )
    
    avg_p, cb = calculate_avg_price_and_cost_basis_single(
        ticker, action, position, price, q_in, old_q, new_q, old_cb
    )

    buyable_sellable = (new_remaining / price) if price > 0 else 0.0
    pv = position_value_from_position(current_position, new_q, price)
    long_u, short_u, u_total = pnl_unrealized_components(new_q, price, avg_p)
    open_pos = open_positions_str()
    open_pv = open_pv_str(ticker, price, current_position)
    open_pnl = open_pnl_unrealized_str(ticker, price, current_position) 

    pv_long_current, pv_short_current = calculate_pv_for_current_ticker(ticker, price, current_position, new_q, avg_p, cb)
    
    total_pv = calculate_total_pv_all_tickers(ticker, price, current_position)
    total_pv_equity = total_pv + new_remaining
    
    # Calculate Total PnL Overall = Equity - Initial Cash
    total_pnl_overall = total_pv_equity - cash
    
    # Calculate Daily PnL = Today's Total PnL Overall - Yesterday's Total PnL Overall
    # Get previous row's Total PnL Overall if it exists, otherwise 0 (first trade)
    previous_df = portfolio_state['portfolio_df']
    if len(previous_df) > 0:
        previous_total_pnl_overall = previous_df.iloc[-1]['Total PnL Overall (Unrealized+Realized)']
        # Get yesterday's equity for Daily % calculation
        yesterday_equity = previous_df.iloc[-1]['Equity: Total PV + Remaining']
    else:
        previous_total_pnl_overall = 0.0
        yesterday_equity = cash  # Use initial cash as baseline for first trade
    
    daily_pnl = total_pnl_overall - previous_total_pnl_overall
    
    # Calculate Daily % = (Today's Equity - Yesterday's Equity) / Yesterday's Equity * 100
    if yesterday_equity > 0:
        daily_pct = ((total_pv_equity - yesterday_equity) / yesterday_equity) * 100
    else:
        daily_pct = None  # Avoid division by zero
    
    # Calculate Cumulative % = ((Equity / Initial Cash) - 1) * 100
    if cash > 0:
        cumulative_pct = ((total_pv_equity / cash) - 1) * 100
    else:
        cumulative_pct = None  # Avoid division by zero

    # Performance is the same as Cumulative %
    performance = cumulative_pct
    
    # Calculate Number of Trades = Track trades per ticker
    # Get trade number for this ticker
    trade_number = get_or_create_trade_number(ticker, old_q, new_q, action)
    
    # Format trade string
    trade_string = format_trade_string(action, current_position, trade_number, new_q)
    
    # Calculate total trades (max trade number that has been assigned)
    if trade_string == "No Buy/Sell":
        # Use previous total if available
        if len(previous_df) > 0:
            last_total = previous_df.iloc[-1].get('Total Trades', "No Buy/Sell")
            if isinstance(last_total, str) and "Trades" in last_total:
                total_trades_str = last_total
            else:
                total_trades_str = "No Buy/Sell"
        else:
            total_trades_str = "No Buy/Sell"
    else:
        # Total trades is the highest trade number opened so far
        total_trades_str = f"{next_trade_number - 1} Trades"
    
    # Calculate Liquidation Price based on current position and quantity
    liquidation_price = calculate_liquidation_price(current_position, new_q, avg_p)
    
    # Calculate Take Profit and Stop Loss
    take_profit = calculate_take_profit(current_position, new_q, avg_p, take_profit_pct)
    stop_loss = calculate_stop_loss(current_position, new_q, avg_p, stop_loss_pct)
    
    # Calculate Win/Loss for closed trades (using trade_string to check if trade closed)
    win_loss = calculate_trade_win_loss(trade_string, realized_pnl, realized_pnl_before)
    
    # Calculate Win Rate at this point
    win_rate = calculate_win_rate(previous_df, win_loss)
    
    # Calculate Win:Loss Ratio at this point
    win_loss_ratio = calculate_win_loss_ratio(previous_df, win_loss)
    
    # Calculate Trades/Month
    trades_per_month = calculate_trades_per_month(previous_df, date, trade_string)
    
    row = {
        'Date': date,
        'Ticker': ticker,
        'Buy/Sell': action.capitalize(),
        'Position Taken': position.capitalize(),
        'Current Position': current_position.capitalize(),
        'Cash': cash,
        'Buyable/Sellable': buyable_sellable,
        'Quantity Buy': q_in,
        'Remaining': new_remaining,
        'Current Quantity': new_q,
        'Price': price,
        'Avg Price': avg_p,
        'Cost Basis': cb,
        'Position Value PV': pv,
        'PnL (Long) Unrealized': long_u,
        'PnL (Short) Unrealized': short_u,
        'Pnl Unrealized': open_pnl,
        'PnL Unrealized Value': u_total,
        'PV (Long)': pv_long_current,
        'PV (Short)': pv_short_current,
        'Open Position': open_pos,
        'Open PV': open_pv,
        'Total PV': total_pv,
        'Equity: Total PV + Remaining': total_pv_equity,
        'PnL Realized': realized_pnl,
        'Total PnL Overall (Unrealized+Realized)': total_pnl_overall,
        'Daily PnL (Unrealized+Realized)': daily_pnl,
        'Liquidation Price': liquidation_price,
        'Take Profit': take_profit,
        'Stop Loss': stop_loss,
        'Last Day Pnl / Daily $': daily_pnl,
        'Daily %': daily_pct,
        'Cumulative %': cumulative_pct,
        'Performance': performance,
        'Trade No. (Position - Trade no. - Current Quantity)': trade_string,
        'Total Trades': total_trades_str,
        'Win/Loss': win_loss,
        'Win Rate': win_rate,
        'Win:Loss Ratio': win_loss_ratio,
        'Trades/Month': trades_per_month
    }

    portfolio_state['remaining'] = new_remaining
    portfolio_state['last_price'][ticker] = price

    df_row = pd.DataFrame([row], columns=COLUMNS)
    portfolio_state['portfolio_df'] = pd.concat(
        [portfolio_state['portfolio_df'], df_row],
        ignore_index=True
    )

    return row

    
def add_trade(ticker, action, position, price, quantity_buy, date=None, take_profit_pct=0.20, stop_loss_pct=0.10):
    process_trade(ticker, action, position, price, quantity_buy, date, take_profit_pct, stop_loss_pct)
    return get_portfolio_df()

In [26]:
reset_portfolio(200)
add_trade('aapl', 'hold', 'long', 10, 0, date='1/1/2025')
add_trade('aapl', 'buy', 'long', 10, 10, date='1/2/2025')
add_trade('aapl', 'hold', 'long', 11, 0, date='1/3/2025')
add_trade('aapl', 'hold', 'long', 12, 0, date='1/4/2025')
add_trade('aapl', 'sell', 'long', 12, 1, date='1/5/2025')
add_trade('aapl', 'hold', 'long', 13, 0, date='1/6/2025')
add_trade('msft', 'buy', 'long', 5, 5, date='1/7/2025')
add_trade('msft', 'hold', 'long', 6, 0, date='2/5/2025')
add_trade('aapl', 'buy', 'long', 13, 2, date='2/6/2025')
add_trade('aapl', 'sell', 'long', 13, 11, date='2/7/2025')
add_trade('aapl', 'sell', 'short', 12, 10, date='2/8/2025')  # Opening short
add_trade('msft', 'sell', 'long', 8, 1, date='2/9/2025')
add_trade('msft', 'buy', 'long', 9, 2, date='3/2/2025')
add_trade('msft', 'sell', 'long', 9, 6, date='3/3/2025')
add_trade('tsla', 'sell', 'short', 20, 5, date='3/4/2025')  # Opening short
add_trade('tsla', 'hold', 'short', 22, 0, date='3/5/2025')
add_trade('tsla', 'buy', 'short', 21, 4, date='3/6/2025')  # Closing short
add_trade('tsla', 'sell', 'short', 22, 1, date='3/7/2025')  # Adding to short
add_trade('tsla', 'buy', 'short', 24, 2, date='3/8/2025')  # Closing short

df = get_portfolio_df()
display(df)

C:\Users\dell\AppData\Local\Temp\ipykernel_13400\2461349179.py:849: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  portfolio_state['portfolio_df'] = pd.concat(


,Date,Ticker,Buy/Sell,Position Taken,Current Position,Cash,Buyable/Sellable,Quantity Buy,Remaining,Current Quantity,Price,Avg Price,Cost Basis,Position Value PV,PnL (Long) Unrealized,PnL (Short) Unrealized,Pnl Unrealized,PnL Unrealized Value,PV (Long),PV (Short),Open Position,Open PV,Total PV,Equity: Total PV + Remaining,PnL Realized,Total PnL Overall (Unrealized+Realized),Daily PnL (Unrealized+Realized),Liquidation Price,Take Profit,Stop Loss,Last Day Pnl / Daily $,Daily %,Cumulative %,Performance,Trade No. (Position - Trade no. - Current Quantity),Total Trades,Win/Loss,Win Rate,Win:Loss Ratio,Trades/Month
0,1/1/2025,AAPL,Hold,Long,Hold,200,20.000000,0.0,200,0,10,0.000000,0.0,0,0.0,0.0,None,0.0,0.0,0.0,None,None,0.0,200.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.000000,0.0,0.0,No Buy/Sell,No Buy/Sell,None,None,0:0,0 (January)
1,1/2/2025,AAPL,Buy,Long,Long,200,10.000000,10.0,100.0,10.0,10,10.000000,100.0,100.0,0.0,0.0,AAPL 0.0,0.0,100.0,0.0,AAPL 10.0,AAPL 100.0,100.0,200.0,0.0,0.0,0.0,0.0,12.000000,9.000000,0.0,0.000000,0.0,0.0,Long - Buy - #1 Trade - 10,1 Trades,None,None,0:0,1 (January)
2,1/3/2025,AAPL,Hold,Long,Long,200,9.090909,0.0,100.0,10.0,11,10.000000,100.0,110.0,10.0,0.0,AAPL 10.0,10.0,110.0,0.0,AAPL 10.0,AAPL 110.0,110.0,210.0,0.0,10.0,10.0,0.0,12.000000,9.000000,10.0,5.000000,5.0,5.0,Long - Hold - #1 Trade - 10,1 Trades,None,None,0:0,1 (January)
3,1/4/2025,AAPL,Hold,Long,Long,200,8.333333,0.0,100.0,10.0,12,10.000000,100.0,120.0,20.0,0.0,AAPL 20.0,20.0,120.0,0.0,AAPL 10.0,AAPL 120.0,120.0,220.0,0.0,20.0,10.0,0.0,12.000000,9.000000,10.0,4.761905,10.0,10.0,Long - Hold - #1 Trade - 10,1 Trades,None,None,0:0,1 (January)
4,1/5/2025,AAPL,Sell,Long,Long,200,9.333333,1.0,112.0,9.0,12,10.000000,90.0,108.0,18.0,0.0,AAPL 18.0,18.0,108.0,0.0,AAPL 9.0,AAPL 108.0,108.0,220.0,2.0,20.0,0.0,0.0,12.000000,9.000000,0.0,0.000000,10.0,10.0,Long - Sell - #1 Trade - 9,1 Trades,None,None,0:0,1 (January)
5,1/6/2025,AAPL,Hold,Long,Long,200,8.615385,0.0,112.0,9.0,13,10.000000,90.0,117.0,27.0,0.0,AAPL 27.0,27.0,117.0,0.0,AAPL 9.0,AAPL 117.0,117.0,229.0,2.0,29.0,9.0,0.0,12.000000,9.000000,9.0,4.090909,14.5,14.5,Long - Hold - #1 Trade - 9,1 Trades,None,None,0:0,1 (January)
6,1/7/2025,MSFT,Buy,Long,Long,200,17.400000,5.0,87.0,5.0,5,5.000000,25.0,25.0,0.0,0.0,"AAPL 27.0, MSFT 0.0",0.0,25.0,0.0,"AAPL 9.0, MSFT 5.0","AAPL 117.0, MSFT 25.0",142.0,229.0,2.0,29.0,0.0,0.0,6.000000,4.500000,0.0,0.000000,14.5,14.5,Long - Buy - #2 Trade - 5,2 Trades,None,None,0:0,2 (January)
7,2/5/2025,MSFT,Hold,Long,Long,200,14.500000,0.0,87.0,5.0,6,5.000000,25.0,30.0,5.0,0.0,"AAPL 27.0, MSFT 5.0",5.0,30.0,0.0,"AAPL 9.0, MSFT 5.0","AAPL 117.0, MSFT 30.0",147.0,234.0,2.0,34.0,5.0,0.0,6.000000,4.500000,5.0,2.183406,17.0,17.0,Long - Hold - #2 Trade - 5,2 Trades,None,None,0:0,1 (February)
8,2/6/2025,AAPL,Buy,Long,Long,200,4.692308,2.0,61.0,11.0,13,10.545455,116.0,143.0,27.0,0.0,"AAPL 27.000000000000007, MSFT 5.0",27.0,143.0,0.0,"AAPL 11.0, MSFT 5.0","AAPL 143.0, MSFT 30.0",173.0,234.0,2.0,34.0,0.0,0.0,12.654545,9.490909,0.0,0.000000,17.0,17.0,Long - Buy - #1 Trade - 11,2 Trades,None,None,0:0,2 (February)
9,2/7/2025,AAPL,Sell,Long,Hold,200,15.692308,11.0,204.0,0.0,13,0.000000,0.0,0.0,0.0,0.0,MSFT 5.0,0.0,0.0,0.0,MSFT 5.0,MSFT 30.0,30.0,234.0,29.0,34.0,0.0,NaN,NaN,NaN,0.0,0.000000,17.0,17.0,Hold - Sell - #1 Trade - 0 - Close,2 Trades,None,None,0:0,2 (February)
